Import Necissary Libraries

In [1]:
import subprocess
import random
import os
import pandas as pd
import shutil

Define Key Variables

In [2]:
#The Mach speed of the airfoil
MACH = 0.5

#The variability of the Mach speed, + or - this number
MACH_VARIABILITY = 0.01

#The list of different alphas run analysis on
ALPHAS = [5, 6]

#The variability of the alphas
ALPHA_VARIABILITY = 0.1

#The airfoil that is loaded into XFoil
AIRFOIL = "NACA 0012"

#Toggle showing the graph UI when running notebook
HIDE_GRAPH = True

#Number of iterations to find convergence
NUM_ITERATIONS = 400

#The Reynolds number
REYNOLDS_NUM = 250000

#Toggle viscous mode as opposed to inviscid mode
VISCOUS_MODE = True

#Number of simulations done using xfoil for each alpha
NUM_SIMS = 100

#Where to save the results
OUTPUT_FILE = "./outputs"

We must define a function to run the commands through XFoil. Because XFoil runs in a console, we use the subprocess library as a proxy to run the commands.

In [3]:
def run_xfoil_commands(command_list):
    """
    Run a batch of XFoil commands using subprocess and return the output.

    Args:
        command_list: The list of commands to run in XFoil

    Returns:
        The console output of XFoil after running commands
    """
    # Join the list of commands into a single string with each command separated by a newline
    commands = "\n".join(command_list)
    
    #Create a subprocess to run the XFoil commands
    process = subprocess.Popen(['./XFOIL6.99/xfoil'],
                               stdin=subprocess.PIPE,
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE,
                               text=True
                               )
    
    #Run the commands and return the console ouput, aswell as any errors
    output, error = process.communicate(input=commands)
    return output

In [4]:
def create_commands(alpha):
    output_path = f"{OUTPUT_FILE}/temp"
    if os.path.exists(output_path):
        shutil.rmtree(output_path)
    os.makedirs(output_path)

    machs = []
    alphas = []

    machs = [round(random.uniform(MACH - MACH_VARIABILITY, MACH + MACH_VARIABILITY), 3) for _ in range(NUM_SIMS)]
    alphas = [round(random.uniform(alpha - ALPHA_VARIABILITY, alpha + ALPHA_VARIABILITY), 3) for _ in range(NUM_SIMS)]

    commands = []

    if HIDE_GRAPH: commands.extend([
        "PLOP",
        "G F",
        ""
    ])
    
    commands.extend([
        AIRFOIL,
        "OPER",
        f"ITER {NUM_ITERATIONS}",
        f"Re {REYNOLDS_NUM}"
    ])

    if VISCOUS_MODE: commands.append("VISC")

    for i in range(NUM_SIMS):
        new_commands = [
            f"MACH {machs[i]}",
            "PACC",
            f"{output_path}/{i}.txt",
            "",
            f"ALFA {alphas[i]}",
            "PACC",
            "PDEL",
            "0"
        ]
        commands.extend(new_commands)
    
    return commands, machs, alphas

In [5]:
def read_output(file_path):
    with open(file_path, 'r') as file:
        for i, line in enumerate(file, start=1):
            if i == 13:  # Check if it's the 13th line
                values = line.split()
                alpha = float(values[0])
                cl = float(values[1])
                cd = float(values[2])
                cdp = float(values[3])
                cm = float(values[4])
                top_xtr = float(values[5])
                bot_xtr = float(values[6])
                return alpha, cl, cd, cdp, cm, top_xtr, bot_xtr
    return None  # Return None if the file doesn't have 13 lines

In [6]:
def process_folder(machs, alphas, folder_path):
    data = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            result = read_output(file_path)
            if result:
                alpha, cl, cd, cdp, cm, top_xtr, bot_xtr = result
                index = int(os.path.splitext(filename)[0])
                mach = machs[index]
                if alpha != alphas[index]: print("Alpha Mismatch!")
                data.append([alpha, mach, cl, cd, cdp, cm, top_xtr, bot_xtr])
    
    # Convert to DataFrame
    df = pd.DataFrame(data, columns=['Alpha', 'Mach', 'CL', 'CD', 'CDp', 'CM', 'Top_Xtr', 'Bot_Xtr'])
    return df

In [7]:
for alpha in ALPHAS:
    folder_path = f"{OUTPUT_FILE}/temp"
    commands, machs, alphas = create_commands(alpha)
    output = run_xfoil_commands(commands)
    df = process_folder(machs, alphas, folder_path)
    shutil.rmtree(folder_path)
    df.to_csv(f"{OUTPUT_FILE}/{alpha}.csv", index=True)